## Work
1. 請改變 reduce_lr 的 patience 和 factor 並比較不同設定下，對訓練/驗證集的影響
2. 請將 optimizer 換成 Adam、RMSprop 搭配 reduce_lr 並比較訓練結果

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
from keras.layers import BatchNormalization

def build_mlp(input_shape, output_units=10, num_neurons=[256, 256, 256]):
    """Code Here
    建立你的神經網路
    """
    return model

def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128]):
    input_layer = keras.layers.Input(input_shape)
    
    for i, n_units in enumerate(num_neurons):
        if i == 0:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(input_layer)
            x = BatchNormalization()(x)
        else:
            x = keras.layers.Dense(units=n_units, 
                                   activation="relu", 
                                   name="hidden_layer"+str(i+1))(x)
            x = BatchNormalization()(x)
    
    out = keras.layers.Dense(units=output_units, activation="softmax", name="output")(x)
    
    model = keras.models.Model(inputs=[input_layer], outputs=[out])
    return model

In [6]:
#LEARNING_RATE = 1e-3
#EPOCHS = 25 # IF you feel too run to finish, try to make it smaller
#BATCH_SIZE = 1024 

LEARNING_RATE = 1e-3
EPOCHS = 20
BATCH_SIZE = 1024
MOMENTUM = 0.95

In [7]:
# 載入 Callbacks
from keras.callbacks import ReduceLROnPlateau

optimizer_set = [keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=0.95),
                 keras.optimizers.Adam(lr=LEARNING_RATE),
                 keras.optimizers.RMSprop(lr=LEARNING_RATE)]

"""Code Here
建立實驗的比較組合
"""
reduce_lr_factor = [0.001,0.0001]
redice_lr_patient = [5,10]

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
import itertools
results = {}
for i, (optim, reduce_factor, reduce_patient) in enumerate(itertools.product(optimizer_set, reduce_lr_factor, redice_lr_patient)):
    print("Numbers of exp: %i, reduce_factor: %.2f, reduce_patient: %i" % (i, reduce_factor, reduce_patient))
    model = build_mlp(input_shape=x_train.shape[1:])
    model.summary()
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optim)
    
    """Code Here
    設定 reduce learning rate 的 callback function
    """
    
    reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=reduce_factor, 
                              monitor='val_loss', 
                              patience=reduce_patient, 
                              verbose=1)
    
    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=BATCH_SIZE, 
              validation_data=(x_test, y_test), 
              shuffle=True,
              callbacks=[reduce_lr]
             )

    # Collect results
    exp_name_tag = ("exp-%s" % (i))
    results[exp_name_tag] = {'train-loss': model.history.history["loss"],
                             'valid-loss': model.history.history["val_loss"],
                             'train-acc': model.history.history["acc"],
                             'valid-acc': model.history.history["val_acc"]}

Numbers of exp: 0, reduce_factor: 0.00, reduce_patient: 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3072)              0         
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
batch_normalization_4 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
batch_normalization_5 (Batch (None, 256)               1024      
_________________________________________________________________
hidden_layer3 (Dense)        (None, 128)               32896     
_________________________________________________________________
batch_normalizatio

50000/50000 [==============================] - 5s 107us/step - loss: 1.1465 - acc: 0.6084 - val_loss: 1.4844 - val_acc: 0.4836
Epoch 17/20
50000/50000 [==============================] - 5s 108us/step - loss: 1.1239 - acc: 0.6152 - val_loss: 1.4714 - val_acc: 0.4857
Epoch 18/20
50000/50000 [==============================] - 5s 107us/step - loss: 1.1014 - acc: 0.6244 - val_loss: 1.4677 - val_acc: 0.4872
Epoch 19/20
50000/50000 [==============================] - 5s 108us/step - loss: 1.0776 - acc: 0.6340 - val_loss: 1.4656 - val_acc: 0.4932
Epoch 20/20
50000/50000 [==============================] - 5s 107us/step - loss: 1.0545 - acc: 0.6432 - val_loss: 1.4633 - val_acc: 0.4899
Numbers of exp: 2, reduce_factor: 0.00, reduce_patient: 5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 3072)              0         
_____________________________________________________________

50000/50000 [==============================] - 6s 111us/step - loss: 1.2233 - acc: 0.5756 - val_loss: 1.4744 - val_acc: 0.4817
Epoch 14/20
50000/50000 [==============================] - 5s 109us/step - loss: 1.1984 - acc: 0.5861 - val_loss: 1.4639 - val_acc: 0.4853
Epoch 15/20
50000/50000 [==============================] - 6s 112us/step - loss: 1.1736 - acc: 0.5933 - val_loss: 1.4627 - val_acc: 0.4847
Epoch 16/20
50000/50000 [==============================] - 5s 109us/step - loss: 1.1512 - acc: 0.6034 - val_loss: 1.4639 - val_acc: 0.4831
Epoch 17/20
50000/50000 [==============================] - 5s 109us/step - loss: 1.1295 - acc: 0.6112 - val_loss: 1.4595 - val_acc: 0.4892
Epoch 18/20
50000/50000 [==============================] - 5s 109us/step - loss: 1.1075 - acc: 0.6203 - val_loss: 1.4422 - val_acc: 0.4943
Epoch 19/20
50000/50000 [==============================] - 5s 109us/step - loss: 1.0844 - acc: 0.6287 - val_loss: 1.4437 - val_acc: 0.4943
Epoch 20/20
50000/50000 [==============

50000/50000 [==============================] - 6s 119us/step - loss: 1.0554 - acc: 0.6303 - val_loss: 1.8268 - val_acc: 0.3816
Epoch 10/20
50000/50000 [==============================] - 6s 118us/step - loss: 1.0213 - acc: 0.6454 - val_loss: 1.5021 - val_acc: 0.4785
Epoch 11/20
50000/50000 [==============================] - 6s 121us/step - loss: 0.9740 - acc: 0.6616 - val_loss: 1.5409 - val_acc: 0.4705
Epoch 12/20
50000/50000 [==============================] - 6s 119us/step - loss: 0.9436 - acc: 0.6720 - val_loss: 1.4968 - val_acc: 0.4823
Epoch 13/20
50000/50000 [==============================] - 6s 119us/step - loss: 0.9028 - acc: 0.6892 - val_loss: 1.5133 - val_acc: 0.4804
Epoch 14/20
50000/50000 [==============================] - 6s 119us/step - loss: 0.8654 - acc: 0.7031 - val_loss: 1.5291 - val_acc: 0.4686
Epoch 15/20
14336/50000 [=======>......................] - ETA: 3s - loss: 0.8130 - acc: 0.7270

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm
import matplotlib.colors as colors
%matplotlib inline
NUM_COLORS = len(results.keys())

cm = plt.get_cmap('gist_rainbow')
cNorm  = colors.Normalize(vmin=0, vmax=NUM_COLORS-1)
scalarMap = mplcm.ScalarMappable(norm=cNorm, cmap=cm)
color_bar = [scalarMap.to_rgba(i) for i in range(NUM_COLORS)]

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label=cond, color=color_bar[i])
plt.title("Loss")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

plt.figure(figsize=(8,6))
for i, cond in enumerate(results.keys()):
    plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label=cond, color=color_bar[i])
    plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label=cond, color=color_bar[i])
plt.title("Accuracy")
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()